# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [52]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

import pandas as pd
import re
from sqlalchemy import create_engine
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [53]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')

# 'message_categories' is the table where I saved the ETL work in the previous workspace
df = pd.read_sql('SELECT * FROM message_categories', engine)

# Using message we want to predict which category the data should fall into
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [54]:
def tokenize(text):
    # Remove punctuation and normalize the text
    new_text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize the text based on word
    tokens = word_tokenize(new_text)
    
    # Remove stop words
    tokens_cleaned = [w for w in tokens if w not in stopwords.words("english")]
    
    # Lemmatize first based on nouns
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in tokens_cleaned]
    # Lemmatize results based on verbs
    final_tokens = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]
    
    return final_tokens
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [55]:
# CountVectorizer Transformer (Bag of Words): Count and vectorize the tokenized words according to "tokenize" method used
# TfidfTransformer Transformer (TF-IDF, Term Frequency-Inverse Document Frequency): Transforms the count matrix to a normalized representation
# MultiOutputClassifier: Multi-target classification
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(LogisticRegression()))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [56]:
# Split data into train/test
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

In [57]:
print("Training data shape: ", X_train.shape)
print("Testing data shape: ", X_test.shape)

Training data shape:  (18351,)
Testing data shape:  (7865,)


In [58]:
print("Training target shape: ", y_train.shape)
print("Testing target shape: ", y_test.shape)

Training target shape:  (18351, 36)
Testing target shape:  (7865, 36)


In [59]:
# Train the pipeline
pipeline.fit(X_train, y_train)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.